<a href="https://colab.research.google.com/github/eggtartplus/nlp/blob/main/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2

In this assignment, we will focus on regular expressions and byte-pair encoding. Assignment 2 is worth 8% toward your final grade.

Should you have any inquiries, please feel free to ask in the GitHub discussion forum found at: https://github.com/orgs/SLPcourse/discussions/categories/assignments.

Please follow the template to finish your assignment and submit in GitHub.


## [50 marks] Regular expression

1. [30marks] Write python code with regular expressions to clean the html webpage.

  Input: https://slpcourse.github.io/materials/html_page.txt

  Reference Output: https://slpcourse.github.io/materials/reference.txt

2. [20 marks] Based on the output from 1, extract the lines with lecture time, tutorial time and office hours. Your need to use regular expressions.


Note: We expect the regular expressions to be just enough for the task. If there are extra non-used regular expressions, we will deduct scores based the lines of non-used regular expressions. Each line that contains non-used regular expressions is worth 5 marks. Each uncleaned html tag is worth 2 points. Each unnecessary whitespace is worth 1 point.

In [121]:
# write your code here
import re
import requests

url = "https://slpcourse.github.io/materials/html_page.txt"
r = requests.get(url)
filename = "html_page.txt"
with open(filename, 'wb') as f:
  f.write(r.content)
with open(filename,'r') as rf:
    con = rf.read()

# removing recurring tags, like <b>,</b>,<a>,</a>
c = re.sub(r'(<b>|<\/b>|<\/a>|<a(.|\n)*?>)','',con)
# spliting the file using <>
c = re.split(r"<[^<>]*>|<!--.*-->",c)
# removing blank
c = [i.strip() for i in c]   #remove leading and trailing spaces
c = [re.sub(r'[\n\t]+', ' ', i) for i in c]
c = list(filter(None, c))   #filter the ""

final = '\n'.join(c)
print(final)

CSC3160 Fundamentals of Speech and Language Processing
MDS6002 Natural Language Processing
Spring 2023
The difference between speech and language processing and other data processing is the use of knowledge of language. In this course, we will study how to describe, process and compute different levels of language knowledge including Phonetics and Phonology, Morphology, Syntax, Semantics, and how the language knowledge is used in speech and language applications such as named entities recognition, information extraction, question answering, speech recognition, and speech synthesis.
Teaching team
Instructor
Zhizheng Wu
TA
Xi Chen
TA
Xueyao Zhang
Poster Session
A final project poster session is planned by the end of the course (tentatively May 20th 2023). This is to provide students the opportunities to connect with speech and language research/industry community.
Anyone from the CUHK-Shenzhen and speech and language technology community are welcome to join. More details will be provided

In [118]:
con1 = re.sub(r'(<b>|<\/b>|<a>|<\/a>)','',con)
pattern_tut = re.compile('.*\d+[:-]\d+.*\n')  # 筛选出所有带有时间的句子，7-9,2:30等 
tut = pattern_tut.findall(con1,re.I|re.S|re.M)
for i in tut:
  new = re.sub(r'(<li>|</li>)','',i)
  print(new.strip())

Lectures: are on Tuesday/Thursday 4:00PM - 5:20PM in TB103. Note: lectures will be remote
Zhizheng Wu: Thu 2:30-3:30 PM. Daoyuan Building 321b
Xi Chen: Wed 7-9PM. SDS Research Lab (4th Floor, Zhi Xin Building) Seat No.100
Xueyao Zhang: Wed 7-9PM. SDS Research Lab (4th Floor, Zhi Xin Building) Seat No.78


## [50 marks] Byte-pair encoding

In Lecture 6, we talked about Byte-Pair Encoding (BPE). In this assignment, the task is to implement a byte-pair encoding algorithm to learn subword tokens.

Here is a vocabulary with frequency, 
```
5 oneumonoultramicroscopicsilicovolcanoconiosis
4 hippopotomonstrosesquippedaliophobia
3 supercalifragilisticexpialidocious
2 pseudopseudohypoparathyroidism
1 floccinaucinihilipilification
2 antidisestablishmentarianism
3 honorificabilitudinitatibus
```
The first column represents the occurency frequency, and the second column represents the words.

In the implementation, k is set to be 5. 

In [103]:
# write your code here
import re, collections

# From the current training data, calculate the frequency of all consecutive character pairs
def get_stats(vocab):
  pairs = collections.defaultdict(int)
  for word, freq in vocab.items():
      symbols = word.split()
      for i in range(len(symbols)-1):
          pairs[symbols[i],symbols[i+1]] += freq
  return pairs

# Use the character pair with the highest frequency to merge the current training data
def merge_vocab(pair, v_in):
  v_out = {}
  bigram = re.escape(' '.join(pair))
  p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
  for word in v_in:
      w_out = p.sub(''.join(pair), word)
      v_out[w_out] = v_in[word]
  return v_out

# Treat the special tag '</w>' as a character to calculate the character length
def len_token(token):
  if token[-4:] == '</w>':
    return len(token[:-4])+1
  else:
    return len(token)

# Generate the current bpe vocabulary based on the current training data
def get_tokens(vocab):
  bpe_vocab = collections.defaultdict(int)
  for word, freq in vocab.items():
      for symbol in word.split():
          bpe_vocab[symbol] += freq
  bpe_vocab = dict(sorted(bpe_vocab.items(), key=lambda x: len_token(x[0]), reverse = True))
  return bpe_vocab


vocab = {'o n e u m o n o u l t r a m i c r o s c o p i c s i l i c o v o l c a n o c o n i o s i s </w>':5, 
         'h i p p o p o t o m o n s t r o s e s q u i p p e d a l i o p h o b i a </w>':4,
         's u p e r c a l i f r a g i l i s t i c e x p i a l i d o c i o u s </w>':3, 
         'p s e u d o p s e u d o h y p o p a r a t h y r o i d i s m </w>':2, 
         'f l o c c i n a u c i n i h i l i p i l i f i c a t i o n </w>':1, 
         'a n t i d i s e s t a b l i s h m e n t a r i a n i s m </w>':2, 
         'h o n o r i f i c a b i l i t u d i n i t a t i b u s </w>':3}

for i in range(5):    # merge times is 5
  pairs = get_stats(vocab)
  best = max(pairs, key = pairs.get)
  if pairs[best] < 2:
    print('Character pair occurrence frequency is less than 2')
    break
  vocab = merge_vocab(best, vocab)
  bpe_vocab = get_tokens(vocab)

print('new training data:{}'.format(vocab))
print('newly generated vocabulary:{}'.format(bpe_vocab.keys()))

new training data:{'on e u m on o u l t r a m ic r os c op ic s i li c o v o l c a n o c on i os i s </w>': 5, 'h i p p op o t o m on s t r os e s q u i p p e d a li op h o b i a </w>': 4, 's u p e r c a li f r a g i li s t ic e x p i a li d o c i o u s </w>': 3, 'p s e u d op s e u d o h y p op a r a t h y r o i d i s m </w>': 2, 'f l o c c i n a u c i n i h i li p i li f ic a t i on </w>': 1, 'a n t i d i s e s t a b li s h m e n t a r i a n i s m </w>': 2, 'h on o r i f ic a b i li t u d i n i t a t i b u s </w>': 3}
newly generated vocabulary:dict_keys(['on', 'ic', 'os', 'op', 'li', 'e', 'u', 'm', 'o', 'l', 't', 'r', 'a', 'c', 's', 'i', 'v', 'n', '</w>', 'h', 'p', 'q', 'd', 'b', 'f', 'g', 'x', 'y'])
